# Declare your model here

In [25]:
from typing import List, Tuple
from mlflow.pyfunc import PythonModel


class ModelTemplate(PythonModel):

    def load_context(self, context) -> None:
      pass

    def predict(self, context, model_input) -> Tuple[List[float],str]:
      pass

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Loading environment variables into the notebook

In [33]:
%dotenv

# Registering the model in MLFlow

In [26]:
import mlflow
import os

model_name = os.environ["MODEL_NAME"]
model_topic_name = os.environ["MODEL_TOPIC"]

def register_model():
  mlflow.set_tracking_uri(os.environ["MLFLOW_CONN_URL"])
  model = ModelTemplate()
  mlflow.pyfunc.log_model(
    artifact_path=os.environ["MLFLOW_BUCKET_NAME"],
    registered_model_name=model_name,
    python_model=model
  )

register_model()

# change the version of the model to production in MLFlow UI

model_path = "models:/"+model_name+"/Production"

MlflowException: Model Registry features are not supported by the store with URI: 'connection-url-of-mlflow-server'. Stores with the following URI schemes are supported: ['databricks', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql'].

# Registering the model in the database

In [ ]:
def register_model_in_database(name, topic):
  import requests
  import json
  token_response = requests.post(os.environ["AUTH_ENDPOINT"])
  response_dict = json.loads(token_response.text)
  token = response_dict["access_token"]

  requests.post(os.environ["MODEL_CREATION_ENDPOINT"], headers={"Authorization: Bearer" + token}, data={"name": name, "publishing_channel": topic})


register_model_in_database(model_name,model_topic_name)

# Cloning and setting up model server production reposityory

In [19]:
from git import Repo

directory = "./" + model_name + "/"

Repo.clone_from("https://github.com/spirabr/SPIRA-Inference-System.git", directory)

<git.repo.base.Repo '/mnt/c/Users/T-Gamer/Documents/USP/SPIRA/ML pipeline/test-1/.git'>

In [22]:
import os

def substitute_occurence_in_file(occurence, replacement, dir, filename, new_filename = ""):
  fin = open(dir + filename, "rt")
  fout = open(dir + "/tmp", "wt")
  for line in fin:
    fout.write(line.replace(occurence, replacement))
  fin.close()
  fout.close()

  if new_filename != "":
    os.remove(dir + filename)
    os.rename(dir + "/tmp",dir + new_filename)
  else:
    os.remove(dir + filename)
    os.rename(dir + "/tmp",dir + filename)

substitute_occurence_in_file('your-model-name-here', model_name, directory,"docker-compose.server.yml")
substitute_occurence_in_file('your-model-name-here', model_name, directory,"build-server.sh")
substitute_occurence_in_file('your-model-name-here', model_name, directory,"stop-server.sh")
substitute_occurence_in_file('/path-to-project/', "/home/spira-inference-system/model-deploys/" + model_name + "/", directory,"docker-compose-model@.service")
substitute_occurence_in_file('your-model-topic-here', model_topic_name, directory,"example.env",new_filename = ".env")
substitute_occurence_in_file('your-model-mlflow-path-here', model_path, directory,".env")

To run the server, create a symbolic link from /etc/systemd/system to docker-compose-model@.service and start it using systemctl.